<a href="https://colab.research.google.com/github/rajat98dogra/LLM_examples/blob/main/LLM_QnA_from_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q  langchain \
                  python-dotenv \
                  tiktoken\
                  faiss-cpu\
                  protobuf \
                  google-generativeai \
                  unstructured

In [ ]:
from langchain.llms import GooglePalm
llm = GooglePalm(google_api_key=key,temperature=.7)

In [ ]:
llm('write peom to PC')

"Oh, PC, my faithful friend,\nYou've always been there for me.\nThrough thick and thin, you've never let me down.\n\nYou've helped me with my work,\nYou've entertained me with games,\nAnd you've always been there to listen to me.\n\nI know I can always count on you,\nAnd I'm so grateful for your friendship.\n\nThank you, PC, for being there for me.\n\nYou're the best friend a person could ask for."

In [ ]:
from langchain.document_loaders import TextLoader, DirectoryLoader

In [ ]:
# docs = TextLoader('./*.txt',delimiter='\n\n').load()
loader = DirectoryLoader(path='./',glob='**/*.txt',loader_cls = TextLoader)

In [ ]:
docs = loader.load()

In [ ]:
len(docs)

2

In [ ]:
! pip install -q transformers

In [ ]:
! pip install -q  InstructorEmbedding  langchain sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.9 MB/s eta 0:00:00


In [ ]:
# creating embedding ----
from langchain.embeddings import HuggingFaceInstructEmbeddings ,HuggingFaceEmbeddings

In [ ]:
embeddings = HuggingFaceInstructEmbeddings()

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
query_result = embeddings.embed_query('well done')


In [ ]:
# creating Vector DB , use for fasten find similar process
from langchain.vectorstores import FAISS

In [ ]:
vector_db = FAISS.from_documents(documents=docs,embedding=embeddings)

In [ ]:
ret = vector_db.as_retriever()


In [ ]:
# ret.get_relevant_documents('GOOGLE Office, Hyderabad ?')

In [ ]:
from langchain.chains import RetrievalQA ,RetrievalQAWithSourcesChain
from langchain.prompts import PromptTemplate

In [ ]:
agent = RetrievalQA.from_chain_type(
                    llm=llm,
                    chain_type='stuff',
                    retriever=ret,
                    input_key='query'
                    )

In [ ]:
agent('Google Office locations?')

{'query': 'Google Office locations?',
 'result': "Google's Hyderabad office is located at Gachibowli, Hyderabad, Telangana, India"}

In [ ]:
agent('percent of skeletons?')

{'query': 'percent of skeletons?', 'result': '45%'}

In [ ]:
Prompt = PromptTemplate(
    template =  """
    Find The Answer from the documents file only.
    \n\n
    {context}

    {question} \n

    """,
    input_variables = ['context','question']
)

In [ ]:
agent = RetrievalQA.from_chain_type(
                    llm=llm,
                    chain_type='stuff',
                    retriever=ret,
                    input_key='query',
                    # return_source_documents=True,
                    chain_type_kwargs = {'prompt':Prompt}
                    )

In [ ]:
agent('percent of skeletons?')


{'query': 'percent of skeletons?', 'result': 'about forty-five percent'}

In [ ]:
agent('locations for internship?')


{'query': 'locations for internship?',
 'result': 'The internship was in Hyderabad.'}